# Reliability: Retry and Fallback Strategies

----

This notebook focuses on **retry and fallback patterns** for building resilient AI applications.

You will learn:

- **Exponential Backoff**: Properly handle transient failures with increasing delays
- **429 Rate Limit Handling**: Respect Retry-After headers and manage quota
- **5xx Error Handling**: Handle server errors gracefully
- **Multi-Backend Fallback**: Route to backup deployments on failure

## Table of Contents

- [Why Retry Strategies Matter](#why-retry-strategies-matter)
- [Setup](#setup)
- [Part 1: Exponential Backoff](#part-1-exponential-backoff)
- [Part 2: 429 Rate Limit Handling](#part-2-429-rate-limit-handling)
- [Part 3: 5xx Error Handling](#part-3-5xx-error-handling)
- [Part 4: Multi-Backend Fallback](#part-4-multi-backend-fallback)
- [Part 5: Complete Resilient Client](#part-5-complete-resilient-client)
- [Best Practices Summary](#best-practices-summary)
- [Wrap-up](#wrap-up)

## Why Retry Strategies Matter

### Common Failure Scenarios

| Error | Cause | Retryable? | Strategy |
|-------|-------|------------|----------|
| **429 Too Many Requests** | Rate limit exceeded | ✅ Yes | Wait for Retry-After |
| **500 Internal Server Error** | Backend issue | ✅ Yes | Exponential backoff |
| **502 Bad Gateway** | Gateway issue | ✅ Yes | Exponential backoff |
| **503 Service Unavailable** | Overloaded | ✅ Yes | Exponential backoff |
| **504 Gateway Timeout** | Timeout | ✅ Yes | Exponential backoff |
| **400 Bad Request** | Invalid input | ❌ No | Fix request |
| **401 Unauthorized** | Invalid credentials | ❌ No | Fix auth |

### The Cost of No Retries

```
Without Retries:           With Smart Retries:
────────────────          ──────────────────
Request → 429 → FAIL      Request → 429 → Wait → Retry → Success
User sees error           User gets response (delayed)
```

## Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [12]:
# Environment setup and PATH configuration
import json
import os
import subprocess
import time
import random
from datetime import datetime
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass
from enum import Enum
from dotenv import load_dotenv

load_dotenv(override=True)

# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
possible_paths = [
    '/opt/homebrew/bin',   # macOS (Apple Silicon)
    '/usr/local/bin',      # macOS (Intel) / Linux
    '/usr/bin',            # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

🔍 Azure CLI found: /anaconda/envs/azureml_py38/bin//az
✅ PATH looks good already

PATH (first 150 chars): /anaconda/envs/azureml_py38/bin/:/afh/code/agent-operator-lab/.venv/bin:/home/azureuser/.vscode-server/cli/servers/Stable-c9d77990917f3102ada88be140d2...


In [13]:
# Load Foundry project settings from .foundry_config.json
from azure.identity import DefaultAzureCredential

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

# Project variables from config
FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
AZURE_AI_PROJECT_ENDPOINT = config.get('AZURE_AI_PROJECT_ENDPOINT')

# Azure OpenAI variables from env
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.environ.get("AZURE_OPENAI_API_VERSION")

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = config.get('AZURE_SUBSCRIPTION_ID', '')

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Azure OpenAI endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"📌 Chat deployment: {AZURE_OPENAI_CHAT_DEPLOYMENT_NAME}")

# Initialize credential for Azure services
credential = DefaultAzureCredential()

✅ Loaded settings from '../0_setup/.foundry_config.json'.

📌 Foundry name: foundry-rq90gs
📌 Resource group: foundry-rg
📌 Location: swedencentral
📌 Azure OpenAI endpoint: https://foundry-rq90gs.cognitiveservices.azure.com/
📌 Chat deployment: gpt-4.1


## Part 1: Exponential Backoff

Exponential backoff increases the delay between retries to avoid overwhelming the service.

In [14]:
# Exponential Backoff implementation

@dataclass
class RetryConfig:
    """Configuration for retry behavior."""
    max_retries: int = 3
    base_delay: float = 1.0  # seconds
    max_delay: float = 60.0  # seconds
    exponential_base: float = 2.0
    jitter: bool = True  # Add randomness to prevent thundering herd

def calculate_backoff_delay(
    attempt: int,
    config: RetryConfig,
    retry_after: Optional[float] = None
) -> float:
    """
    Calculate delay before next retry using exponential backoff.
    
    Formula: min(base_delay * (exponential_base ^ attempt), max_delay)
    With jitter: delay * (0.5 + random(0, 0.5))
    """
    # If Retry-After header provided, use it
    if retry_after is not None:
        return retry_after
    
    # Calculate exponential delay
    delay = config.base_delay * (config.exponential_base ** attempt)
    delay = min(delay, config.max_delay)
    
    # Add jitter to prevent thundering herd
    if config.jitter:
        jitter_factor = 0.5 + random.random() * 0.5  # 0.5 to 1.0
        delay *= jitter_factor
    
    return delay

# Demonstrate exponential backoff delays
print("📊 Exponential Backoff Demonstration")
print("=" * 60)

config = RetryConfig(max_retries=5, base_delay=1.0, max_delay=60.0)

print(f"\n⚙️ Configuration:")
print(f"   Base delay: {config.base_delay}s")
print(f"   Max delay: {config.max_delay}s")
print(f"   Exponential base: {config.exponential_base}")
print(f"   Jitter: {config.jitter}")

print(f"\n📈 Delay progression (without jitter):")
config_no_jitter = RetryConfig(jitter=False)
for attempt in range(6):
    delay = calculate_backoff_delay(attempt, config_no_jitter)
    bar = "█" * int(delay / 2)
    print(f"   Attempt {attempt}: {delay:5.1f}s │ {bar}")

print(f"\n📈 Delay progression (with jitter):")
for attempt in range(6):
    delay = calculate_backoff_delay(attempt, config)
    bar = "█" * int(delay / 2)
    print(f"   Attempt {attempt}: {delay:5.1f}s │ {bar}")

📊 Exponential Backoff Demonstration

⚙️ Configuration:
   Base delay: 1.0s
   Max delay: 60.0s
   Exponential base: 2.0
   Jitter: True

📈 Delay progression (without jitter):
   Attempt 0:   1.0s │ 
   Attempt 1:   2.0s │ █
   Attempt 2:   4.0s │ ██
   Attempt 3:   8.0s │ ████
   Attempt 4:  16.0s │ ████████
   Attempt 5:  32.0s │ ████████████████

📈 Delay progression (with jitter):
   Attempt 0:   0.9s │ 
   Attempt 1:   1.6s │ 
   Attempt 2:   4.0s │ █
   Attempt 3:   6.2s │ ███
   Attempt 4:  15.0s │ ███████
   Attempt 5:  24.1s │ ████████████


## Part 2: 429 Rate Limit Handling with fake_response

Handle rate limits properly by respecting Retry-After headers.

In [15]:
# 429 Rate Limit Handler
from openai import AzureOpenAI, RateLimitError, APIStatusError
import httpx

@dataclass
class RetryResult:
    """Result of a retry operation."""
    success: bool
    response: Optional[str]
    attempts: int
    total_delay: float
    errors: List[str]

def extract_retry_after(error: Exception) -> Optional[float]:
    """Extract Retry-After value from error response."""
    if hasattr(error, 'response') and error.response is not None:
        headers = getattr(error.response, 'headers', {})
        retry_after = (
            headers.get('Retry-After')
            or headers.get('retry-after')
        )
        if retry_after:
            try:
                return float(retry_after)
            except ValueError:
                pass
    return None

def is_retryable_error(error: Exception) -> bool:
    """Check if an error is retryable."""
    if isinstance(error, RateLimitError):
        return True
    if isinstance(error, APIStatusError):
        status_code = getattr(error, 'status_code', 0)
        return status_code in [429, 500, 502, 503, 504]
    return False

def call_with_retry(
    func: Callable,
    config: RetryConfig = RetryConfig()
) -> RetryResult:
    """Execute a function with retry logic."""
    attempts = 0
    total_delay = 0
    errors = []

    while attempts <= config.max_retries:
        try:
            result = func()
            return RetryResult(
                success=True,
                response=result,
                attempts=attempts + 1,
                total_delay=total_delay,
                errors=errors,
            )
        except Exception as e:
            errors.append(
                f"Attempt {attempts + 1}: "
                f"{type(e).__name__}: {str(e)[:100]}"
            )

            if not is_retryable_error(e):
                return RetryResult(
                    success=False,
                    response=None,
                    attempts=attempts + 1,
                    total_delay=total_delay,
                    errors=errors,
                )

            if attempts >= config.max_retries:
                break

            retry_after = extract_retry_after(e)
            delay = calculate_backoff_delay(
                attempts, config, retry_after
            )

            print(
                f"   ⏳ Retry {attempts + 1}"
                f"/{config.max_retries} "
                f"after {delay:.1f}s..."
            )
            time.sleep(delay)
            total_delay += delay
            attempts += 1

    return RetryResult(
        success=False,
        response=None,
        attempts=attempts,
        total_delay=total_delay,
        errors=errors,
    )


# ── Forced-failure demo ──────────────────────────────────────
# We inject a counter so the first N calls throw a *real*
# RateLimitError, then the final call goes to the actual API.

class FaultInjector:
    """Wraps an API call and forces failures for the
    first `fail_count` invocations."""

    def __init__(self, real_func: Callable, fail_count: int = 2):
        self.real_func = real_func
        self.fail_count = fail_count
        self.call_number = 0

    def __call__(self):
        self.call_number += 1
        if self.call_number <= self.fail_count:
            # Build a realistic 429 response object
            fake_response = httpx.Response(
                status_code=429,
                headers={"Retry-After": "1"},
                json={
                    "error": {
                        "code": "429",
                        "message": "Rate limit exceeded "
                        "(simulated)",
                    }
                },
                request=httpx.Request("POST", "https://fake"),
            )
            raise RateLimitError(
                message=(
                    f"[Simulated] Rate limit hit "
                    f"(call {self.call_number})"
                ),
                response=fake_response,
                body=None,
            )
        # After enough failures, call the real API
        return self.real_func()


print("📊 Retry Demo – forced 429 errors then real success")
print("=" * 60)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
    )

    def real_request():
        resp = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {"role": "user", "content": "Say 'ok'"}
            ],
            max_tokens=10,
        )
        return resp.choices[0].message.content

    # First 2 calls → simulated 429, 3rd call → real API
    injector = FaultInjector(real_request, fail_count=2)
    result = call_with_retry(
        injector,
        RetryConfig(max_retries=3, base_delay=0.5),
    )

    print(f"\n{'✅' if result.success else '❌'} Result:")
    print(f"   Success:     {result.success}")
    print(f"   Response:    {result.response}")
    print(f"   Attempts:    {result.attempts}")
    print(f"   Total delay: {result.total_delay:.1f}s")
    if result.errors:
        print(f"   Errors hit:  {len(result.errors)}")
        for e in result.errors:
            print(f"      • {e}")
else:
    print(
        "⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 Retry Demo – forced 429 errors then real success
   ⏳ Retry 1/3 after 1.0s...


   ⏳ Retry 2/3 after 1.0s...

✅ Result:
   Success:     True
   Response:    ok
   Attempts:    3
   Total delay: 2.0s
   Errors hit:  2
      • Attempt 1: RateLimitError: [Simulated] Rate limit hit (call 1)
      • Attempt 2: RateLimitError: [Simulated] Rate limit hit (call 2)


## Part 3: 5xx Error Handling with fake 5xx errors

Handle server-side errors with appropriate retry strategies.

In [16]:
# 5xx Error Handler with classification

class ErrorType(Enum):
    """Classification of API errors."""
    RATE_LIMIT = "rate_limit"      # 429
    SERVER_ERROR = "server_error"  # 500, 502, 503, 504
    CLIENT_ERROR = "client_error"  # 400, 401, 403
    TIMEOUT = "timeout"            # Connection/read timeout
    UNKNOWN = "unknown"

def classify_error(error: Exception) -> ErrorType:
    """Classify an error for appropriate handling."""
    if isinstance(error, RateLimitError):
        return ErrorType.RATE_LIMIT
    if isinstance(error, APIStatusError):
        status = getattr(error, 'status_code', 0)
        if status == 429:
            return ErrorType.RATE_LIMIT
        elif status >= 500:
            return ErrorType.SERVER_ERROR
        elif status >= 400:
            return ErrorType.CLIENT_ERROR
    if 'timeout' in str(error).lower():
        return ErrorType.TIMEOUT
    return ErrorType.UNKNOWN

def get_retry_config_for_error(
    error_type: ErrorType,
) -> Optional[RetryConfig]:
    """Get appropriate retry config for each error type."""
    configs = {
        ErrorType.RATE_LIMIT: RetryConfig(
            max_retries=5, base_delay=1.0, max_delay=60.0,
        ),
        ErrorType.SERVER_ERROR: RetryConfig(
            max_retries=3, base_delay=2.0, max_delay=30.0,
        ),
        ErrorType.TIMEOUT: RetryConfig(
            max_retries=2, base_delay=1.0, max_delay=10.0,
        ),
        ErrorType.CLIENT_ERROR: None,
        ErrorType.UNKNOWN: RetryConfig(
            max_retries=1, base_delay=1.0,
        ),
    }
    return configs.get(error_type)


# ── Live demo: force different error types ────────────────
class Multi5xxInjector:
    """Simulates a sequence of 5xx errors followed by
    a real successful API call."""

    ERROR_SEQUENCE = [502, 503, 500]  # rotate through these

    def __init__(self, real_func: Callable, fail_count: int = 2):
        self.real_func = real_func
        self.fail_count = fail_count
        self.call_number = 0

    def __call__(self):
        self.call_number += 1
        if self.call_number <= self.fail_count:
            code = self.ERROR_SEQUENCE[
                (self.call_number - 1) % len(self.ERROR_SEQUENCE)
            ]
            fake = httpx.Response(
                status_code=code,
                json={
                    "error": {
                        "code": str(code),
                        "message": f"Simulated {code}",
                    }
                },
                request=httpx.Request("POST", "https://fake"),
            )
            raise APIStatusError(
                message=f"[Simulated] {code} error",
                response=fake,
                body=None,
            )
        return self.real_func()


print("📊 5xx Error Classification + Retry Demo")
print("=" * 60)

# 1️⃣  Show error classification table
print("\n📋 Error Types and Retry Strategies:")
print("-" * 60)
for error_type in ErrorType:
    cfg = get_retry_config_for_error(error_type)
    if cfg:
        print(
            f"   {error_type.value.upper():15s} │ "
            f"retries={cfg.max_retries}  "
            f"base={cfg.base_delay}s  "
            f"max={cfg.max_delay}s"
        )
    else:
        print(
            f"   {error_type.value.upper():15s} │ "
            f"❌ No retry (fix the request)"
        )

# 2️⃣  Live run: inject 502 → 503 → then real call
if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    print("\n\n🔥 Live run: simulating 502 → 503 → real call")
    print("-" * 60)

    def real_5xx_request():
        resp = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {
                    "role": "user",
                    "content": "Say 'recovered!'",
                }
            ],
            max_tokens=10,
        )
        return resp.choices[0].message.content

    injector_5xx = Multi5xxInjector(
        real_5xx_request, fail_count=2
    )

    server_cfg = get_retry_config_for_error(
        ErrorType.SERVER_ERROR
    )
    server_cfg.base_delay = 0.3  # speed up for demo

    result = call_with_retry(injector_5xx, server_cfg)

    print(f"\n{'✅' if result.success else '❌'} Result:")
    print(f"   Success:     {result.success}")
    print(f"   Response:    {result.response}")
    print(f"   Attempts:    {result.attempts}")
    print(f"   Total delay: {result.total_delay:.1f}s")
    if result.errors:
        print(f"   Errors hit:")
        for e in result.errors:
            # Classify each error for demonstration
            print(f"      • {e}")
else:
    print(
        "\n⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 5xx Error Classification + Retry Demo

📋 Error Types and Retry Strategies:
------------------------------------------------------------
   RATE_LIMIT      │ retries=5  base=1.0s  max=60.0s
   SERVER_ERROR    │ retries=3  base=2.0s  max=30.0s
   CLIENT_ERROR    │ ❌ No retry (fix the request)
   TIMEOUT         │ retries=2  base=1.0s  max=10.0s
   UNKNOWN         │ retries=1  base=1.0s  max=60.0s


🔥 Live run: simulating 502 → 503 → real call
------------------------------------------------------------
   ⏳ Retry 1/3 after 0.3s...


   ⏳ Retry 2/3 after 0.5s...

✅ Result:
   Success:     True
   Response:    recovered!
   Attempts:    3
   Total delay: 0.8s
   Errors hit:
      • Attempt 1: APIStatusError: [Simulated] 502 error
      • Attempt 2: APIStatusError: [Simulated] 503 error


## Part 4: Multi-Backend Fallback with non-existent-deployment-xyz

Route to backup deployments when primary fails.

In [17]:
# Multi-Backend Fallback implementation

@dataclass
class BackendConfig:
    """Configuration for a backend deployment."""
    name: str
    endpoint: str
    api_key: str
    deployment: str
    priority: int = 1  # Lower = higher priority

@dataclass
class FallbackResult:
    """Result of a fallback operation."""
    success: bool
    response: Optional[str]
    backend_used: str
    backends_tried: List[str]
    total_attempts: int

class FallbackClient:
    """Client with multi-backend fallback support."""

    def __init__(self, backends: List[BackendConfig]):
        self.backends = sorted(
            backends, key=lambda b: b.priority
        )
        self.clients: Dict[str, AzureOpenAI] = {}
        for backend in self.backends:
            self.clients[backend.name] = AzureOpenAI(
                azure_endpoint=backend.endpoint,
                api_key=backend.api_key,
                api_version=AZURE_OPENAI_API_VERSION,
            )

    def complete(
        self,
        messages: List[Dict[str, str]],
        retry_config: RetryConfig = RetryConfig(),
        **kwargs,
    ) -> FallbackResult:
        """Make a completion call with fallback support."""
        backends_tried = []
        total_attempts = 0

        for backend in self.backends:
            backends_tried.append(backend.name)
            client = self.clients[backend.name]

            print(f"   🔄 Trying backend: {backend.name}")

            def make_request(
                _client=client, _backend=backend
            ):
                return _client.chat.completions.create(
                    model=_backend.deployment,
                    messages=messages,
                    **kwargs,
                )

            result = call_with_retry(
                make_request, retry_config
            )
            total_attempts += result.attempts

            if result.success:
                return FallbackResult(
                    success=True,
                    response=(
                        result.response.choices[0]
                        .message.content
                    ),
                    backend_used=backend.name,
                    backends_tried=backends_tried,
                    total_attempts=total_attempts,
                )

            print(
                f"   ❌ Backend {backend.name} failed "
                f"after {result.attempts} attempts"
            )
            for err in result.errors:
                print(f"      • {err}")

        return FallbackResult(
            success=False,
            response=None,
            backend_used="none",
            backends_tried=backends_tried,
            total_attempts=total_attempts,
        )


# ── Demo: first backend is DELIBERATELY broken ──────────
print("📊 Multi-Backend Fallback Demo")
print("=" * 60)
print(
    "Scenario: primary backend has a BAD deployment name "
    "→ should fallback to secondary\n"
)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    backends = [
        # ❌ Primary – wrong deployment → will fail with 404
        BackendConfig(
            name="primary-BROKEN",
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            deployment="non-existent-deployment-xyz",
            priority=1,
        ),
        # ✅ Secondary – correct deployment
        BackendConfig(
            name="secondary-OK",
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            priority=2,
        ),
    ]

    fallback_client = FallbackClient(backends)

    result = fallback_client.complete(
        messages=[
            {
                "role": "user",
                "content": "Say 'hello from fallback'",
            }
        ],
        retry_config=RetryConfig(
            max_retries=1, base_delay=0.3
        ),
        max_tokens=20,
    )

    print(f"\n{'✅' if result.success else '❌'} Result:")
    print(f"   Success:        {result.success}")
    print(f"   Response:       {result.response}")
    print(f"   Backend used:   {result.backend_used}")
    print(f"   Backends tried: {result.backends_tried}")
    print(f"   Total attempts: {result.total_attempts}")
else:
    print(
        "⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 Multi-Backend Fallback Demo
Scenario: primary backend has a BAD deployment name → should fallback to secondary

   🔄 Trying backend: primary-BROKEN
   ❌ Backend primary-BROKEN failed after 1 attempts
      • Attempt 1: NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this r
   🔄 Trying backend: secondary-OK



✅ Result:
   Success:        True
   Response:       hello from fallback
   Backend used:   secondary-OK
   Backends tried: ['primary-BROKEN', 'secondary-OK']
   Total attempts: 2


## Part 5: Complete Resilient Client with fault-injected 429 / 502 errors

Combine all strategies into a production-ready client.

In [19]:
# Complete Resilient Client

class ResilientOpenAIClient:
    """
    Production-ready OpenAI client with:
    - Exponential backoff
    - Error classification
    - Multi-backend fallback
    - Comprehensive logging
    """

    def __init__(
        self,
        backends: List[BackendConfig],
        default_retry_config: RetryConfig = RetryConfig(),
    ):
        self.fallback_client = FallbackClient(backends)
        self.default_retry_config = default_retry_config
        self.stats = {
            "total_requests": 0,
            "successful_requests": 0,
            "failed_requests": 0,
            "total_retries": 0,
            "fallbacks_used": 0,
        }

    def complete(
        self,
        messages: List[Dict[str, str]],
        **kwargs,
    ) -> FallbackResult:
        """Make a resilient completion call."""
        self.stats["total_requests"] += 1

        result = self.fallback_client.complete(
            messages=messages,
            retry_config=self.default_retry_config,
            **kwargs,
        )

        if result.success:
            self.stats["successful_requests"] += 1
        else:
            self.stats["failed_requests"] += 1
        self.stats["total_retries"] += (
            result.total_attempts - 1
        )
        if len(result.backends_tried) > 1:
            self.stats["fallbacks_used"] += 1

        return result

    def get_stats(self) -> Dict[str, Any]:
        """Get client statistics."""
        stats = self.stats.copy()
        if stats["total_requests"] > 0:
            stats["success_rate"] = (
                stats["successful_requests"]
                / stats["total_requests"]
                * 100
            )
            stats["avg_retries"] = (
                stats["total_retries"]
                / stats["total_requests"]
            )
        return stats


# ── Demo: mixed failure scenarios ─────────────────────────
# We send 5 queries through the resilient client.
# Queries 2 and 4 get fault-injected 429 errors that
# require retries, while the others succeed on first try.

print("📊 Resilient Client – Mixed Failure Demo")
print("=" * 60)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    backends = [
        BackendConfig(
            name="primary",
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            priority=1,
        ),
    ]

    resilient_client = ResilientOpenAIClient(
        backends=backends,
        default_retry_config=RetryConfig(
            max_retries=3, base_delay=0.3
        ),
    )

    queries = [
        ("What is 2+2?", 0),      # clean
        ("Capital of France?", 2), # 2× forced 429
        ("Say 'test'", 0),         # clean
        ("Color of the sky?", 1),  # 1× forced 502
        ("Say 'done'", 0),         # clean
    ]

    print("\n📝 Sending 5 queries (some will be "
          "fault-injected)…\n")

    for idx, (query, n_faults) in enumerate(queries, 1):
        label = (
            f"🔥 {n_faults} injected fault(s)"
            if n_faults
            else "🟢 clean call"
        )
        print(f"── Query {idx}: \"{query}\"  ({label})")

        # Monkey-patch the fallback client so its
        # internal make_request goes through the injector
        original_complete = (
            resilient_client.fallback_client.complete
        )

        if n_faults:
            # Wrap with injector
            def patched_complete(
                messages,
                retry_config=RetryConfig(),
                _nf=n_faults,
                **kw,
            ):
                backend = (
                    resilient_client.fallback_client
                    .backends[0]
                )
                _client = (
                    resilient_client.fallback_client
                    .clients[backend.name]
                )

                def real_fn():
                    return _client.chat.completions.create(
                        model=backend.deployment,
                        messages=messages,
                        **kw,
                    )

                injector = FaultInjector(
                    real_fn, fail_count=_nf
                )
                r = call_with_retry(injector, retry_config)
                return FallbackResult(
                    success=r.success,
                    response=(
                        r.response.choices[0]
                        .message.content
                        if r.success else None
                    ),
                    backend_used=backend.name,
                    backends_tried=[backend.name],
                    total_attempts=r.attempts,
                )

            resilient_client.fallback_client.complete = (
                patched_complete
            )
        else:
            resilient_client.fallback_client.complete = (
                original_complete
            )

        result = resilient_client.complete(
            messages=[
                {"role": "user", "content": query}
            ],
            max_tokens=30,
        )

        # Restore original
        resilient_client.fallback_client.complete = (
            original_complete
        )

        icon = "✅" if result.success else "❌"
        snippet = (
            result.response[:50]
            if result.response else "N/A"
        )
        print(
            f"   {icon} attempts={result.total_attempts}  "
            f"→ {snippet}\n"
        )

    # Display stats
    stats = resilient_client.get_stats()
    print("=" * 60)
    print("📈 Client Statistics:")
    print(f"   Total requests:  {stats['total_requests']}")
    print(f"   Successful:      "
          f"{stats['successful_requests']}")
    print(f"   Failed:          {stats['failed_requests']}")
    print(f"   Success rate:    "
          f"{stats.get('success_rate', 0):.1f}%")
    print(f"   Total retries:   {stats['total_retries']}")
    print(f"   Fallbacks used:  {stats['fallbacks_used']}")
else:
    print(
        "⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 Resilient Client – Mixed Failure Demo

📝 Sending 5 queries (some will be fault-injected)…

── Query 1: "What is 2+2?"  (🟢 clean call)
   🔄 Trying backend: primary


   ✅ attempts=1  → 2 + 2 = 4

── Query 2: "Capital of France?"  (🔥 2 injected fault(s))
   ⏳ Retry 1/3 after 1.0s...
   ⏳ Retry 2/3 after 1.0s...
   ✅ attempts=3  → The capital of France is **Paris**.

── Query 3: "Say 'test'"  (🟢 clean call)
   🔄 Trying backend: primary
   ✅ attempts=1  → test

── Query 4: "Color of the sky?"  (🔥 1 injected fault(s))
   ⏳ Retry 1/3 after 1.0s...
   ✅ attempts=2  → The color of the sky is **usually blue** during th

── Query 5: "Say 'done'"  (🟢 clean call)
   🔄 Trying backend: primary
   ✅ attempts=1  → done

📈 Client Statistics:
   Total requests:  5
   Successful:      5
   Failed:          0
   Success rate:    100.0%
   Total retries:   3
   Fallbacks used:  0


## Best Practices Summary

### Retry Strategy Guidelines

| Scenario | Max Retries | Base Delay | Strategy |
|----------|-------------|------------|----------|
| Rate limits (429) | 5 | 1s | Exponential + Retry-After |
| Server errors (5xx) | 3 | 2s | Exponential + jitter |
| Timeouts | 2 | 1s | Quick retry |
| Client errors (4xx) | 0 | N/A | Don't retry, fix request |

### Code-Level Fallback vs API Gateway Routing

| Aspect | Code-Level Fallback (this notebook) | API Gateway Routing (e.g. APIM) |
|--------|--------------------------------------|--------------------------------|
| **Where it lives** | Inside application code | Infrastructure (gateway) layer |
| **Deployment changes** | Code change + redeploy required | Policy update — instant, no redeploy |
| **Language dependency** | Must implement per language (Python, JS, …) | Language-agnostic — transparent to all clients |
| **Fine-grained control** | ✅ Custom logic per request | ⚠️ Applied uniformly via policy rules |
| **Observability** | Application logs only | Centralized dashboard & metrics |
| **Complexity at scale** | Risk of duplicated logic across services | Single place for consistent policy management |
| **Latency overhead** | Direct call — no extra hop | Gateway hop — slight added latency |
| **Cost** | No additional infrastructure | Gateway service cost (e.g. APIM tier) |
| **Operational ownership** | Owned by each dev team | Managed centrally by platform team |

> **Recommendation**: For prototypes or single-service apps, **Code-Level** fallback is fast and simple.
> In production, where multiple services share the same AI backends, centralizing retry/fallback policies in an **API Gateway** (e.g. Azure API Management) is more operationally efficient.
> You can also **combine both** — let the gateway handle primary retry/routing, and keep application-level fallback for business-logic-specific scenarios.

### Architecture Recommendations

```
┌─────────────────────────────────────────────────────────────────┐
│                    Resilient AI Client                          │
├─────────────────────────────────────────────────────────────────┤
│  1. Error Classification (retryable vs non-retryable)          │
│  2. Exponential Backoff with Jitter                            │
│  3. Retry-After Header Respect                                  │
│  4. Multi-Backend Fallback (Primary → Secondary → Tertiary)    │
│  5. Comprehensive Metrics/Logging                               │
└─────────────────────────────────────────────────────────────────┘
```

## Wrap-up

### What You Learned

1. **Exponential Backoff**: Implement proper retry delays with jitter
2. **429 Handling**: Respect rate limits and Retry-After headers
3. **5xx Handling**: Classify and handle server errors appropriately
4. **Multi-Backend Fallback**: Route to backup deployments on failure
5. **Resilient Client**: Combine strategies for production use

### Additional Resources

- [Azure OpenAI Rate Limits](https://learn.microsoft.com/en-us/azure/ai-services/openai/quotas-limits)
- [Retry Pattern - Cloud Design Patterns](https://learn.microsoft.com/en-us/azure/architecture/patterns/retry)
- [Circuit Breaker Pattern](https://learn.microsoft.com/en-us/azure/architecture/patterns/circuit-breaker)